In [3]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from tkinterdnd2 import DND_FILES, TkinterDnD
import threading
import datetime
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
MAX_SECONDS_ALLOWED = 5        # タイムアウト短縮（5秒）
ERROR_WAIT_TIME = 60           # トークン枯渇時の待機（60秒）
SAVE_INTERVAL = 10             # 10件ごとに保存
STOP_FLAG = False

# ============================================================
# Keepa API呼び出し（軽量・節約モード）
# ============================================================
def fetch_price(api_key, jan):
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={jan}"
        "&history=0&offers=1&onlyLiveOffers=1&buybox=1&stats=0"
    )
    try:
        resp = requests.get(url, timeout=MAX_SECONDS_ALLOWED)
        if resp.status_code == 429:
            return None, "トークン枯渇"
        data = resp.json()
    except Exception as e:
        return None, f"通信エラー: {e}"

    tokens_left = data.get("tokensLeft", 100)
    if tokens_left < 30:
        return None, "トークン残少"

    if "products" not in data or not data["products"]:
        return None, "商品が見つからない"

    p = data["products"][0]
    title = p.get("title", "")
    stats = p.get("stats") or {}

    # ✅ BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return (title, int(v)), None

    # ✅ Primeまたは最初の出品
    offers = p.get("offers") or []
    if offers:
        o = next((o for o in offers if o.get("isPrime")), offers[0])
        price = o.get("price") or 0
        ship = o.get("shipping") or 0
        if price > 0:
            return (title, price + ship), None

    return (title, "Null"), "価格取得失敗"

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_btn):
    global STOP_FLAG
    STOP_FLAG = False
    start_btn.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("エラー", f"Excelを開けませんでした:\n{e}")
        start_btn.config(state="normal")
        return

    total = len(df)
    results = []
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = os.path.join(os.path.expanduser("~"), "Desktop", f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 全{total}件の処理を開始\n")

    for i, row in df.iterrows():
        if STOP_FLAG:
            break

        jan = str(row.iloc[0]).strip()
        if not jan or jan.lower() == "nan":
            continue

        result, error = fetch_price(api_key, jan)
        if error:
            if "トークン枯渇" in error or "トークン残少" in error:
                log_box.insert(tk.END, f"🪙 {i+1}/{total} {jan} → {error}。60秒待機\n")
                log_box.see(tk.END)
                time.sleep(ERROR_WAIT_TIME)
                continue
            log_box.insert(tk.END, f"⚠️ {i+1}/{total} {jan} → {error}\n")
            log_box.see(tk.END)

        title, price = result if result else ("", "Null")
        results.append({"JANコード": jan, "商品名": title, "価格": price, "備考": error or ""})

        log_box.insert(tk.END, f"✅ {i+1}/{total} 件完了\n")
        log_box.see(tk.END)

        # 10件ごとに保存
        if (i + 1) % SAVE_INTERVAL == 0:
            pd.DataFrame(results).to_excel(out_path, index=False)
            log_box.insert(tk.END, f"💾 {i+1}件保存\n")
            log_box.see(tk.END)

    pd.DataFrame(results).to_excel(out_path, index=False)
    messagebox.showinfo("完了", f"処理が完了しました！\n結果: {out_path}")
    start_btn.config(state="normal")

# ============================================================
# GUI（最軽量DnD対応）
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（DnD軽量版）")
    root.geometry("380x260")
    root.configure(bg="#faf7f2")
    root.resizable(False, False)

    tk.Label(root, text="Keepa APIキー：", bg="#faf7f2").pack(anchor="w", padx=10, pady=(5,0))
    api_entry = tk.Entry(root, width=50, show="*")
    api_entry.pack(padx=10, pady=(0,5))

    tk.Label(root, text="Excelファイル（ドラッグ＆ドロップ対応）：", bg="#faf7f2").pack(anchor="w", padx=10)
    file_label = tk.Label(root, text="ここにファイルをドロップ", bg="white",
                          width=45, height=2, relief="groove")
    file_label.pack(padx=10, pady=5)
    file_label.filepath = None

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.filepath = filepath
        file_label.config(text=os.path.basename(filepath))

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)

    log_box = tk.Text(root, height=8, width=45)
    log_box.pack(padx=10, pady=5)

    def stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "🛑 停止しました\n")
        log_box.see(tk.END)

    start_btn = tk.Button(root, text="▶ 開始", bg="#4CAF50", fg="white",
        command=lambda: threading.Thread(
            target=start_process,
            args=(api_entry.get().strip(), file_label.filepath, log_box, start_btn),
            daemon=True
        ).start())
    start_btn.pack(side="left", padx=30, pady=5)

    tk.Button(root, text="■ 停止", bg="#d9534f", fg="white", command=stop).pack(side="right", padx=30, pady=5)
    root.mainloop()

if __name__ == "__main__":
    create_gui()
